# VacationPy
---

- Module 6 Challenge: "In this deliverable, you'll use your weather data skills to plan future vacations. Also, you'll use Jupyter notebooks, the geoViews Python library, and the Geoapify API."
  
- The following notebook was developed from the provided starter code.
  
- Student name: Steph Abegg 

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [4]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Convert city names to title case (wasn't required but it looks nicer)
city_data_df["City"] = [city_name.title() for city_name in city_data_df["City"]]

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Ushuaia,-54.8000,-68.3000,3.81,60,0,8.23,AR,1721705991
1,1,Port Saint John'S,-31.6229,29.5448,12.76,78,6,2.98,ZA,1721705992
2,2,Puerto Natales,-51.7236,-72.4875,3.52,93,100,2.02,CL,1721705994
3,3,Udachny,66.4167,112.4000,16.70,36,36,4.13,RU,1721705995
4,4,Bilibino,68.0546,166.4372,26.15,48,46,7.17,RU,1721705997


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [46]:
%%capture --no-display

# Configure the map plot
city_map = city_data_df.hvplot.points("Lng", 
                                      "Lat", 
                                      geo = True,
                                      color = "City",
                                      alpha = 0.6,
                                      frame_width = 800,
                                      frame_height = 600,
                                      size = "Humidity",
                                      tiles = "OSM",
                                      title = "Cities in Dataset, Sized by Humidity"
                                     )

# Display the map
city_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [6]:
# Narrow down cities that fit criteria and drop any results with null values

# My Criteria:
# A max temperature higher than 20 degrees
# Humidity < 50%
# Cloudiness < 30%
# Wind speed less than 2.2 m/s (5mph)

ideal_city_data_df = city_data_df[(city_data_df["Max Temp"] > 20) 
                                  & (city_data_df["Humidity"] < 50) 
                                  & (city_data_df["Cloudiness"] < 30) 
                                  & (city_data_df["Wind Speed"] < 2.2) 
                                 #  & (city_data_df["Country"] == "US")
                                 ]
# Drop any rows with null values
ideal_city_data_df = ideal_city_data_df.dropna(how="any")

# Display sample data
ideal_city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
10,10,Pocone,-16.2567,-56.6228,20.15,36,4,1.38,BR,1721706004
26,26,Whitehorse,60.7161,-135.0538,22.43,43,20,2.06,CA,1721705769
101,101,Zeya,53.7500,127.2667,30.63,34,5,0.84,RU,1721706119
116,116,Fuerte Olimpo,-21.0415,-57.8738,20.42,37,13,0.75,PY,1721706138
178,178,Yoloten,37.2989,62.3597,34.36,18,0,0.72,TM,1721706219
327,327,Pul-E 'Alam,33.9953,69.0227,26.96,35,3,0.65,AF,1721706408
329,329,Mary,37.0000,62.5000,34.46,18,0,0.67,TM,1721706411
343,343,Ardestan,33.3761,52.3694,29.88,29,0,1.71,IR,1721706430
371,371,Sar-E Pul,35.8333,66.1667,25.54,24,0,1.51,AF,1721706464
408,408,Zhangjiakou,40.8100,114.8794,33.07,39,0,2.13,CN,1721706512


### Step 3: Create a new DataFrame called `hotel_df`.

In [7]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_city_data_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df


,City,Country,Lat,Lng,Humidity,Hotel Name
10,Pocone,BR,-16.2567,-56.6228,36,
26,Whitehorse,CA,60.7161,-135.0538,43,
101,Zeya,RU,53.7500,127.2667,34,
116,Fuerte Olimpo,PY,-21.0415,-57.8738,37,
178,Yoloten,TM,37.2989,62.3597,18,
327,Pul-E 'Alam,AF,33.9953,69.0227,35,
329,Mary,TM,37.0000,62.5000,18,
343,Ardestan,IR,33.3761,52.3694,29,
371,Sar-E Pul,AF,35.8333,66.1667,24,
408,Zhangjiakou,CN,40.8100,114.8794,39,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [8]:
# Set parameters to search for a hotel
radius = 10000 # hotel located within 10,000m of city coordinates
params = {"categories": "accommodation.hotel", # find hotels
          "apiKey": geoapify_key 
         }

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}" # Search places inside of the circle
    params["bias"] = f"proximity:{lng},{lat}" # Search first near the location	

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params = params)

    # Convert the API response to JSON format
    name_address = name_address.json()
    #display(name_address)

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
Pocone - nearest hotel: Andirah
Whitehorse - nearest hotel: Town & Mountain Hotel
Zeya - nearest hotel: Серебряный створ
Fuerte Olimpo - nearest hotel: No hotel found
Yoloten - nearest hotel: No hotel found
Pul-E 'Alam - nearest hotel: No hotel found
Mary - nearest hotel: No hotel found
Ardestan - nearest hotel: Tourism Hotel
Sar-E Pul - nearest hotel: No hotel found
Zhangjiakou - nearest hotel: 商务宾馆
Aripuana - nearest hotel: No hotel found
Dinar - nearest hotel: Otel Asya
Canutama - nearest hotel: No hotel found
Al Hufuf - nearest hotel: Al Muhaidab Residence Al Ahsa
Sao Felix Do Xingu - nearest hotel: Hotel Terraço
Cukurca - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
10,Pocone,BR,-16.2567,-56.6228,36,Andirah
26,Whitehorse,CA,60.7161,-135.0538,43,Town & Mountain Hotel
101,Zeya,RU,53.7500,127.2667,34,Серебряный створ
116,Fuerte Olimpo,PY,-21.0415,-57.8738,37,No hotel found
178,Yoloten,TM,37.2989,62.3597,18,No hotel found
327,Pul-E 'Alam,AF,33.9953,69.0227,35,No hotel found
329,Mary,TM,37.0000,62.5000,18,No hotel found
343,Ardestan,IR,33.3761,52.3694,29,Tourism Hotel
371,Sar-E Pul,AF,35.8333,66.1667,24,No hotel found
408,Zhangjiakou,CN,40.8100,114.8794,39,商务宾馆


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [40]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points("Lng", 
                                   "Lat", 
                                   geo = True,
                                   color = "City",
                                   alpha = 0.8,
                                   size = 200, # makes less sense to soze by humidity now
                                   frame_width = 800,
                                   frame_height = 500,
                                   tiles = "OSM",
                                   hover_cols = ["Hotel Name", "Country"],
                                   title = "Steph's Ideal Cities to Visit (Warm, Low Humidity, Sunny, and Calm)"
                                  )

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)

**Discussion:** 

The above map shows all of Steph's ideal places to visit in the set of random cities. Some of these places do not have hotels within 10 km. Steph want's to have a hotel so she can continue to work remotely and have a place to be at night. So the code below filters the dataframe to just places with hotels within 10 km, and again plots the map.

Also, because I had already filtered to cities with a low humidity and because temperature and cloudiness are also important to my criteria, it makes less sense to size by humidity. So I made all of the markers the same size.

In [48]:
%%capture --no-display

# Configure the map plot
hotel_map_2 = hotel_df[hotel_df["Hotel Name"]!="No hotel found"].hvplot.points(
                                   "Lng", 
                                   "Lat", 
                                   geo = True,
                                   color = "City",
                                   alpha = 0.8,
                                   size = 200, # makes less sense to soze by humidity now
                                   frame_width = 800,
                                   frame_height = 500,
                                   tiles = "OSM",
                                   hover_cols = ["Hotel Name", "Country"],
                                   title = "Steph's Ideal Cities to Visit (Warm, Low Humidity, Sunny, and Calm), \
                                   \nShowing Only Places With Hotels Within 10 km of City Coordinates"
                                  )

# Display the map
hotel_map_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)